In [1]:
import pyarabic.araby as araby
import pyarabic.number as number
import string
import re
import pickle
from collections import Counter

In [60]:
# letter_to_id = {}
# id_to_letter = {}
# word_to_id = {}
# id_to_word = {}
# diacritic_to_id = {}
# id_to_diacritic = {}
# word_count = Counter()

# id_letter = 0
# id_word = 0
# id_diacritic = 1

# diacritic_to_id["space"] = 0
# id_to_diacritic[0] = "space"

with open('Dataset_with_shaddah/test_cleaned_withshadda_215.txt','r',encoding='utf-8') as file:
    for line in file:
        letters, diacritics =araby.separate(line)
        for index,letter in enumerate(letters):
            if (letter == '\n') or (letter == '\u200f'):
                continue
                
            if letter == 'ّ':
                if diacritics[index] != 'ـ' :
                    shaddad_with_diac = letter+diacritics[index]
                    if shaddad_with_diac not in diacritic_to_id:
                        diacritic_to_id[shaddad_with_diac] = id_diacritic
                        id_to_diacritic[id_diacritic] = shaddad_with_diac
                        id_diacritic += 1
                else:
                    if letter not in diacritic_to_id:
                        diacritic_to_id[letter] = id_diacritic
                        id_to_diacritic[id_diacritic] = letter
                        id_diacritic += 1
            else:
                if letter not in letter_to_id:
                    letter_to_id[letter] = id_letter
                    id_to_letter[id_letter] = letter
                    id_letter += 1
                    
                diacritic = diacritics[index]
                if diacritic not in diacritic_to_id:
                    diacritic_to_id[diacritic] = id_diacritic
                    id_to_diacritic[id_diacritic] = diacritic
                    id_diacritic += 1
                    
        words = araby.tokenize(line)
        for word in words:
            if word == '\n':
                continue
            word_count[word] += 1
            
            if word not in word_to_id:
                word_to_id[word] = id_word
                id_to_word[id_word] = word
                id_word +=1
                
     
        

In [61]:
with open('Dataset_with_shaddah/letter_to_id.pickle', 'wb') as file:
    pickle.dump(letter_to_id, file)
    
with open('Dataset_with_shaddah/id_to_letter.pickle', 'wb') as file:
    pickle.dump(id_to_letter, file)

In [62]:
with open('Dataset_with_shaddah/word_to_id.pickle', 'wb') as file:
    pickle.dump(word_to_id, file)
    
with open('Dataset_with_shaddah/id_to_word.pickle', 'wb') as file:
    pickle.dump(id_to_word, file)

In [63]:
with open('Dataset_with_shaddah/diacritic_to_id.pickle', 'wb') as file:
    pickle.dump(diacritic_to_id, file)
    
with open('Dataset_with_shaddah/id_to_diacritic.pickle', 'wb') as file:
    pickle.dump(id_to_diacritic, file)

In [64]:
with open('Dataset_with_shaddah/word_count.pickle', 'wb') as file:
    pickle.dump(word_count, file)

In [86]:
with open('Dataset_with_shaddah/id_to_word.pickle', 'rb') as file:
    with_shaddah = pickle.load(file)
len(with_shaddah)

86782

In [87]:
with open('Dataset/id_to_word.pickle', 'rb') as file:
    no_shaddah = pickle.load(file)
len(no_shaddah)

86529